In [5]:
import psycopg2


In [6]:
import getpass
import os

postgres_url = getpass.getpass("Enter your postgresql url: ")

## csv

In [26]:
import psycopg2
import csv
import pandas as pd
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

cur.execute("SET search_path TO travel_database, public;")
query=f'SELECT * FROM hotel'
#get list duplicate
df=pd.read_sql_query(query,conn)
df.to_csv('hotel.csv', index=False)
cur.close()
conn.close()
print('copied')

/tmp/ipykernel_8708/441506653.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query,conn)


copied


## deduplicate

In [27]:
#name 
import psycopg2
import csv
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

cur.execute("SET search_path TO travel_database, public;")

#get list duplicate
cur.execute("""
SELECT hotel_id, name, location
FROM hotel h1
WHERE (
    SELECT COUNT(*)
    FROM hotel h2
    WHERE h2.name = h1.name
      AND h2.location = h1.location
) > 1
ORDER BY name, location, hotel_id;
""")

duplicates= cur.fetchall()
cur.close()
conn.close()
print(len(duplicates))
csv_file = "duplicate_hotels_name.csv"

    # Write to CSV file
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
        # Write header
    writer.writerow(["ID", "Name",'Location'])
        # Write rows
    writer.writerows(duplicates)

    print(f"Duplicate hotels have been written to {csv_file}")


0
Duplicate hotels have been written to duplicate_hotels_name.csv


In [20]:
#delete duplicate hotel and room
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

cur.execute("SET search_path TO travel_database, public;")

#get list duplicate
delete_rooms_query = """
    WITH duplicate_hotels AS (
        SELECT hotel_id
        FROM hotel
        WHERE (name, location) IN (
            SELECT name, location
            FROM hotel
            GROUP BY name, location
            HAVING COUNT(*) > 1
        )
    )
    DELETE FROM hotelprice
    WHERE hotel_id IN (SELECT hotel_id FROM duplicate_hotels);
    """
    
cur.execute(delete_rooms_query)
    
    # Step 2: Delete the duplicate hotels, keeping only the first (lowest hotel_id)
delete_hotels_query = """
    DELETE FROM hotel
    WHERE hotel_id IN (
        SELECT hotel_id
        FROM (
            SELECT hotel_id, 
                   ROW_NUMBER() OVER (PARTITION BY name, location ORDER BY hotel_id) AS row_num
            FROM hotel
        ) AS subquery
        WHERE row_num > 1
    );
    """
    
cur.execute(delete_hotels_query)

    # Commit the changes
conn.commit()

print("Duplicate hotels and their associated rooms have been deleted, keeping the first hotel.")

Duplicate hotels and their associated rooms have been deleted, keeping the first hotel.


In [22]:
import psycopg2
import csv
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

cur.execute("SET search_path TO travel_database, public;")

#get list duplicate
cur.execute("""
SELECT hotel_id, name, location
FROM hotel h1
WHERE (
    SELECT COUNT(*)
    FROM hotel h2 
    WHERE h2.location = h1.location
) > 1
ORDER BY location, hotel_id;
""")

duplicates= cur.fetchall()
cur.close()
conn.close()
# Specify the CSV file name
csv_file = "duplicate_hotels.csv"

    # Write to CSV file
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
        # Write header
    writer.writerow(["ID", "Name", "Location"])
        # Write rows
    writer.writerows(duplicates)

    print(f"Duplicate hotels have been written to {csv_file}")

print(len(duplicates))



Duplicate hotels have been written to duplicate_hotels.csv
641


## amenities


In [5]:
import pandas as pd

conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

cur.execute("SET search_path TO travel_database, public;")

cur.execute("""
    SELECT DISTINCT unnest(amenities) AS unique_amenity
    FROM hotel;
""")

rows = cur.fetchall()

cur.close()
conn.close()

amenities_list = [row[0] for row in rows]
amenities_list_str = "\n    ".join(f'"{amenity}"' for amenity in amenities_list)
print(amenities_list_str)

"WiFi tại khu vực chung"
    "Money changer"
    "Bathtub"
    "Tiện nghi cho trẻ"
    "Quầy bar bên hồ bơi"
    "Heater"
    "Dù (ô) che nắng"
    "Đưa đón đến khu trượt tuyết (thu phí)"
    "Tiện nghi hội họp"
    "Sân quần vợt ngoài trời"
    "Trung tâm chăm sóc trẻ em"
    "Roll-in shower"
    "Vegetarian meal"
    "Hồ bơi"
    "Tủ lạnh (dùng chung)"
    "Máy photocopy"
    "Giữ trẻ"
    "Porter"
    "Bicycle storage"
    "Dịch vụ cho thuê xe đạp"
    "Conference room"
    "Dịch vụ trông trẻ có người giám hộ"
    "Grocery"
    "Giặt ủi"
    "Dịch vụ phòng 24 giờ"
    "Lò vi sóng"
    "Express check-out"
    "Đưa đón sân bay"
    "AC"
    "Dịch vụ phòng (có giới hạn thời gian)"
    "Bóng quần"
    "TV lounge"
    "A la carte lunch"
    "Bồn tắm nước nóng"
    "Gói cầu hôn lãng mạn"
    "Safety deposit box"
    "Vườn thú bán hoang dã"
    "Hồ bơi trẻ em"
    "Ghế dài tắm nắng"
    "Đưa đón đến trạm xe buýt (thu phí)"
    "Wifi (miễn phí)"
    "Thẩm mỹ viện"
    "Bữa trưa món tự chọn"